**Flowsheet Containing a Single Mixer**

**Problem Statement**: Consider the process of mixing benzene and toluene streams to form a mixture with the following inlet specificatons:

**Stream 1:**
Component: 

Benzene Flow Rate = 0.6 kmol/hr

Pressure = 101325 Pa 

Temperature = 353 K

**Stream 2**
Component: 

Toluene Flow Rate = 0.4 kmol/hr

Pressure = 202650 Pa 

Temperature = 356 K

We will study the simulation of the mixer with the given specifications using 2 approaches:

Case 1: Specifying the number of mixer inlets, and momentum mixing type set to default ('minimize')

Case 2: Specifying the mixer inlet names, and momentum mixing type set to 'equality'

For more details, IDAES documentation reference for mixer model: https://idaes-pse.readthedocs.io/en/stable/models/mixer.html

**Case 1**
**Setting up the problem in IDAES**

In [1]:
# Import pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

#Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
# Steady State Model

#import the BTX property package to create a properties block for the flowsheet
from idaes.generic_models.properties.activity_coeff_models import BTX_activity_coeff_VLE

# Add properties as parameters to flowsheet directly, and attach the properties block to the flowsheet
m.fs.properties = BTX_activity_coeff_VLE.BTXParameterBlock(default={"valid_phase":
                                                     'Liq',
                                                     "activity_coeff_model":
                                                     "Ideal"})
#Import a mixer unit
from idaes.generic_models.unit_models.mixer import Mixer

#Create an instance of the mixer unit, attaching it to the flowsheet
#specify that the property package to be used with the mixer is the one we created earlier.
m.fs.mixer1 = Mixer(default={"property_package": m.fs.properties,"num_inlets":2})

#import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom
# DOF = Number of Model Variables - Number of Model Constraints

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

#Fix the stream inlet conditions
m.fs.mixer1.inlet_1.flow_mol.fix(0.6*1000/3600)
m.fs.mixer1.inlet_1.mole_frac_comp[0, "benzene"].fix(1)
m.fs.mixer1.inlet_1.mole_frac_comp[0, "toluene"].fix(0)
m.fs.mixer1.inlet_1.pressure.fix(101325)
m.fs.mixer1.inlet_1.temperature.fix(353)
m.fs.mixer1.inlet_2.flow_mol.fix(0.4*1000/3600)
m.fs.mixer1.inlet_2.mole_frac_comp[0, "benzene"].fix(0)
m.fs.mixer1.inlet_2.mole_frac_comp[0, "toluene"].fix(1)
m.fs.mixer1.inlet_2.pressure.fix(202650)
m.fs.mixer1.inlet_2.temperature.fix(356)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom are: {0}'.format(DOF_final))

The initial degrees of freedom are: 10
The final degrees of freedom are: 0


**Obtain Simulation Results**

In [2]:
#Initialize the flowsheet, and set the output at WARNING
m.fs.mixer1.initialize(outlvl=idaeslog.WARNING)

#Solve the simulation using ipopt
#Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
result = opt.solve(m)

In [3]:
#Display a readable report
m.fs.mixer1.report()


Unit : fs.mixer1                                                           Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                             inlet_1    inlet_2    Outlet  
    flow_mol                  0.16667    0.11111    0.27778
    mole_frac_comp benzene     1.0000     0.0000    0.60000
    mole_frac_comp toluene     0.0000     1.0000    0.40000
    temperature                353.00     356.00     354.31
    pressure               1.0132e+05 2.0265e+05 1.0132e+05


In [4]:
#Display the outlet stream molar enthalpy (J/mol)
m.fs.mixer1.mixed_state[0].enth_mol_phase.display()

enth_mol_phase : Phase molar specific enthalpies [J/mol]
    Size=1, Index=fs.properties.phase_list
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    Liq :  None : 42703.868530442516 :  None : False : False :  Reals


In [13]:
#Energy Balance Validation 
molflow_1 = m.fs.mixer1.inlet_1.flow_mol[0].value
molflow_2 = m.fs.mixer1.inlet_2.flow_mol[0].value
benzene_1 = m.fs.mixer1.inlet_1.mole_frac_comp[0, "benzene"].value
toluene_2 = m.fs.mixer1.inlet_2.mole_frac_comp[0, "toluene"].value
temp_1 = m.fs.mixer1.inlet_1.temperature[0].value
temp_2 = m.fs.mixer1.inlet_2.temperature[0].value

# sum(m*cp*(t-tref))_i = outlet enthalpy
cp_benzene = 133 #J/mol.K
cp_toluene = 157 #J/mol.K

#total_enthalpy_in = molflow_1*benzene_1*cp_benzene*(temp_1) + molflow_2*toluene_2*cp_toluene*(temp_2)
#m.fs.mixer1.inlet_1_state.display()
total_enthalpy_in1 = molflow_1*m.fs.mixer1.inlet_1_state[0].enth_mol_phase['Liq'].value
total_enthalpy_in2 = molflow_2*m.fs.mixer1.inlet_2_state[0].enth_mol_phase['Liq'].value
print(total_enthalpy_in1 + total_enthalpy_in2)

#molar_enthalpy_out = m.fs.mixer1.mixed_state[0].enth_mol_phase.value
molar_enthalpy_out = 42703.868530442516
total_entalpy_out = (molflow_1 + molflow_2)*molar_enthalpy_out
print(total_entalpy_out)

11862.1857029007
11862.185702900699


**Case 2**

In [14]:
#Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m1 = ConcreteModel()
m1.fs = FlowsheetBlock(default={"dynamic": False})
# Steady State Model

# Add properties as parameters to flowsheet directly, and attach the properties block to the flowsheet
m1.fs.properties = BTX_activity_coeff_VLE.BTXParameterBlock(default={"valid_phase":
                                                     'Liq',
                                                     "activity_coeff_model":
                                                     "Ideal"})

In [15]:
#Create an instance of another mixer unit, attaching it to the flowsheet
#specify that the property package to be used with the mixer is the one we created earlier.
m1.fs.mixer2 = Mixer(default={"property_package": m.fs.properties,"inlet_list":["inlet1","inlet2"],"momentum_mixing_type":2})

In [16]:
#Fix the stream inlet conditions
m1.fs.mixer2.inlet1.flow_mol.fix(0.6*1000/3600)
m1.fs.mixer2.inlet1.mole_frac_comp[0, "benzene"].fix(1)
m1.fs.mixer2.inlet1.mole_frac_comp[0, "toluene"].fix(0)
m1.fs.mixer2.inlet1.pressure.fix(101325)
m1.fs.mixer2.inlet1.temperature.fix(353)
m1.fs.mixer2.inlet2.flow_mol.fix(0.4*1000/3600)
m1.fs.mixer2.inlet2.mole_frac_comp[0, "benzene"].fix(0)
m1.fs.mixer2.inlet2.mole_frac_comp[0, "toluene"].fix(1)
m1.fs.mixer2.inlet2.pressure.fix(202650)
m1.fs.mixer2.inlet2.temperature.fix(356)

In [17]:
#Initialize the flowsheet, and set the output at WARNING
m1.fs.mixer2.initialize(outlvl=idaeslog.INFO)

2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet1_state: Initialization Step 1 skipped.
2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet1_state: Initialization Step 2 optimal - Optimal Solution Found.
2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet1_state: Initialization Step 3 optimal - Optimal Solution Found.
2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet1_state: Initialization Step 4 optimal - Optimal Solution Found.
2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet1_state: Initialization Step 5 optimal - Optimal Solution Found.
2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet2_state: Initialization Step 1 skipped.
2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet2_state: Initialization Step 2 optimal - Optimal Solution Found.
2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet2_state: Initialization Step 3 optimal - Optimal Solution Found.
2020-04-09 13:53:58 [INFO] idaes.init.fs.mixer2.inlet2_state: Initialization Step 4 optimal - Optimal Solu

In [18]:
#Solve the simulation using ipopt
#Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt',tee=True)
result = opt.solve(m1)

    model=unknown;
        message from solver=Too few degrees of freedom (rethrown)!


In [ ]:
#Display a readable report
m1.fs.mixer2.report()